In [232]:
import glob
import numpy as np
import scipy as sp
import einops as eo
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
import magpylib as magpy
import os



M0 = 1450 #mT
shape = [3*25.4/16, 25.4/8] #diameter and height
x0 = np.array([0,0,10, 0,0,1])

In [233]:
def B_dipole(position, rotation, M0, shape):
    R = np.sqrt(np.sum(position**2, axis=1))
    B = (M0 * (shape[0]) ** 2 * shape[1] / (16)) * (
        (
            3
            * position
            / R[:, np.newaxis] ** 5
            * (eo.einsum(position, rotation, "sensor dim,  dim -> sensor"))[
                :, np.newaxis
            ]
        )
        - rotation[np.newaxis, :] / (R[:, np.newaxis] ** 3)
    )
    return B


def getField_dipole(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def getField_dipole_fixed(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def cost_dipole(x, B, positions, M0, shape):
    diff = getField_dipole(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)


def getField_cylinder(x, positions, M0, shape):
    B=magpy.getB(
        sources="Cylinder",
        position=x[:3],
        orientation=Rotation.align_vectors(x[3:], np.array([0, 0, 1]))[0],
        observers=positions,
        dimension=shape,
        polarization=(0, 0, M0),
    )
    return B

def cost_cylinder(x, B, positions, M0, shape):
    diff = getField_cylinder(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)

def minimize(x0, B, positions, M0, shape, *args):
    #print("Starting mimimization")
    b_args = (B, positions, M0, shape)
    cons = [{"type": "eq", "fun": lambda x: x[3] ** 2 + x[4] ** 2 + x[5] ** 2 - 1}]
    bounds = [(-100, 100), (-100, 100), (0, 100), (-1, 1), (-1, 1), (-1, 1)]
    res = sp.optimize.minimize(
        fun=cost_cylinder, x0=x0, args=b_args, tol=1e-100, constraints=cons, bounds=bounds, *args
    ).x  
    #print(f"Finished mimimization with shape {b_args[3]} at {res}")
    return res

def circle_radius(x1, y1, x2, y2, x3, y3):
    # Calculate lengths of sides of the triangle formed by the three points
    a = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    b = math.sqrt((x3 - x2) ** 2 + (y3 - y2) ** 2)
    c = math.sqrt((x3 - x1) ** 2 + (y3 - y1) ** 2)

    # Calculate the semi-perimeter of the triangle
    s = (a + b + c) / 2

    # Calculate the area of the triangle using Heron's formula
    area = math.sqrt(s * (s - a) * (s - b) * (s - c))

    # Calculate the radius of the circle using the formula: radius = (abc) / (4 * area)
    radius = (a * b * c) / (4 * area)
    
    return radius

def comparison_plot(locact, locpred, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    # Plotting
    plt.figure(figsize=(8, 6))
    plt.scatter(index, pred, label='Predicted' + rel)
    plt.scatter(index, act, label='Actual' + rel)
    plt.xlabel('Index')
    plt.ylabel(axis + '-component')
    plt.title('Comparison of '+axis+ ' Location' + rel+ ': Predicted vs Actual')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def comp_plot3D(locact, locpred, xact, yact, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    fig = go.Figure(data=[
        go.Scatter3d(x=xact, y=yact, z=pred, name='Predicted' + rel),
        go.Scatter3d(x=xact, y=yact, z=act, name='Actual' + rel)])
    fig.show()

    return

def displacement_plot(actual, predicted):

    dispx = np.array(predicted[:,0]-predicted[:,0][0])
    dispy = np.array(predicted[:,1]-predicted[:,1][0])
    dispz = np.array(predicted[:,2]-predicted[:,2][0])
    disp = np.sqrt(dispx**2+dispy**2+dispz**2)
    actdispx = np.array(actual[:,0]-actual[:,0][0])
    actdispy = np.array(actual[:,1]-actual[:,1][0])
    actdispz = np.array(actual[:,2]-actual[:,2][0])
    actdisp = np.sqrt(actdispx**2+actdispy**2+actdispz**2)

    plt.figure(figsize=(8, 6))
    plt.scatter(list(range(len(disp))), disp, label='Predicted Displacement')
    plt.scatter(list(range(len(actdisp))), actdisp, label='Actual Displacement')
    plt.xlabel('Index')
    plt.ylabel('Displacement (mm)')
    plt.title('Magnet Displacement')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def importfitting(folder, indices):

    file_list = sorted(glob.glob(f'{folder}/*.npz'))
    print(len(file_list))
    data = np.load(file_list[0])
    pos = data['pos'][0]

    locpred = [np.array([0,0,0])]
    anglepred = [np.array([0,0,0])]
    Bstdev = [np.array(pos)]
    for x in range(len(file_list)):
        data = np.load(file_list[x])
        mags = data['mags']
        pos = data['pos'][0][indices]
        B = np.mean(mags[0][indices], axis=0)
        mags_stdev = np.std(mags, axis=0)
        x_res = minimize(x0, B, pos, M0, shape)
        locpred = np.append(locpred, [x_res[0:3]], axis=0)
        anglepred = np.append(anglepred, [x_res[3:6]], axis=0)
        Bstdev = np.append(Bstdev, [mags_stdev], axis=0)
    locpred = np.delete(locpred, 0, axis=0)
    anglepred = np.delete(anglepred, 0, axis=0)
    Bstdev = np.delete(Bstdev, 0 ,axis=0)

    return pos, mags, locpred, anglepred, Bstdev

def actualangle(xind, yind, xscal, xcon, yscal, ycon, isoff):

    angact = [np.array([0, 0, 0])]
    for y in range(yind):
        for x in range(xind):
            R = Rotation.from_euler('ZYX' ,np.array([0, yscal*y-ycon, xscal*x-xcon]), degrees=True)
            norm = np.array([[0, 0, 1]])
            rotated = R.apply(norm, inverse=False)[0]
            #add = np.array([np.sin(np.deg2rad(2*y-4)), -np.sin(np.deg2rad(2*x-4)), np.cos(np.deg2rad(2*y-4))*np.cos(np.deg2rad(2*x-4))])
            angact = np.append(angact, [rotated], axis=0)
    angact = np.delete(angact, 0, axis=0)
    locact = angact*isoff

    return angact, locact

In [248]:
folder = os.path.join('..', './MLX07172024_D32N52_AboutZ_5mm7')
pos, mags, locpred, anglepred, Bstdev = importfitting(folder, range(16))
np.append(locpred, anglepred, axis=1)


13
[[-0.02649532 -4.25241179 16.67554275]
 [ 2.18878041 -4.20199621 16.75561748]
 [ 4.52354473 -2.82951108 16.86566827]
 [ 6.17283183 -0.76854943 16.88695572]
 [ 6.12697138  1.43381398 16.96713378]
 [ 4.35403261  3.7137322  17.06596161]
 [ 2.09678787  5.17215316 17.00034078]
 [-0.23113232  5.25201062 16.9032004 ]
 [-2.12499985  3.64160171 16.85416948]
 [-3.43518945  1.43181288 16.76403589]
 [-3.3675663  -0.81861735 16.69479641]
 [-2.08935464 -2.83864234 16.68207897]
 [-0.04497233 -4.24004778 16.67953677]]


In [250]:
print(locpred)

[[-0.02649532 -4.25241179 16.67554275]
 [ 2.18878041 -4.20199621 16.75561748]
 [ 4.52354473 -2.82951108 16.86566827]
 [ 6.17283183 -0.76854943 16.88695572]
 [ 6.12697138  1.43381398 16.96713378]
 [ 4.35403261  3.7137322  17.06596161]
 [ 2.09678787  5.17215316 17.00034078]
 [-0.23113232  5.25201062 16.9032004 ]
 [-2.12499985  3.64160171 16.85416948]
 [-3.43518945  1.43181288 16.76403589]
 [-3.3675663  -0.81861735 16.69479641]
 [-2.08935464 -2.83864234 16.68207897]
 [-0.04497233 -4.24004778 16.67953677]]


In [249]:
print(anglepred)

[[-4.84015176e-04 -1.03004351e-01  9.94680788e-01]
 [ 4.16692180e-02 -9.71568489e-02  9.94396411e-01]
 [ 1.02893471e-01 -5.38656219e-02  9.93232817e-01]
 [ 1.60231986e-01 -1.19776566e-02  9.87006710e-01]
 [ 1.55992788e-01  2.26659692e-02  9.87498103e-01]
 [ 9.24544110e-02  7.34569364e-02  9.93003656e-01]
 [ 3.62691325e-02  1.22916026e-01  9.91754103e-01]
 [-3.80949541e-03  1.30297637e-01  9.91467606e-01]
 [-4.10911054e-02  8.08922509e-02  9.95875477e-01]
 [-7.78066056e-02  2.76273918e-02  9.96585601e-01]
 [-7.75467848e-02 -1.57715343e-02  9.96863960e-01]
 [-4.29633241e-02 -6.20426678e-02  9.97148364e-01]
 [-8.13592280e-04 -1.02595377e-01  9.94722839e-01]]


In [235]:
file_list = sorted(glob.glob(f'{folder}/*.npz'))
len(file_list)
data = np.load(file_list[0])
pos = data['pos'][0]

all_mags = [np.array([0,0,0])]
for x in range(len(file_list)):
    data = np.load(file_list[x])
    mags = data['mags']
    all_mags = np.append(all_mags, mags[0], axis=0)
    pos = data['pos'][0]
    B = np.mean(mags, axis=0)
    mags_stdev = np.std(mags, axis=0)
all_mags = np.delete(all_mags, 0, axis=0)

print(all_mags)
bx = all_mags[:,0]
by = all_mags[:,1]
bz = all_mags[:,2]

print(bx)
len(bx)

[[-2.4072    1.0014    2.573428]
 [-1.2363    1.6065    5.112492]
 [ 1.4331    1.5864    4.87146 ]
 [ 2.289     0.9318    2.396284]
 [-2.5317   -0.6663    2.797036]
 [-1.344    -1.209     5.394664]
 [ 1.5672   -1.2039    5.486624]
 [ 2.4489   -0.6939    2.57246 ]
 [-1.7673   -1.6866    1.670284]
 [-0.8667   -2.6088    3.050168]
 [ 0.9273   -2.487     2.93304 ]
 [ 1.5936   -1.5441    1.52944 ]
 [-0.8853   -1.3791    0.535788]
 [-0.4563   -1.9047    0.977196]
 [ 0.3531   -1.9191    0.94138 ]
 [ 0.7917   -1.3461    0.512556]
 [-2.1348    1.416     3.965896]
 [ 0.2076    1.728     5.504532]
 [ 2.2581    1.2492    3.587892]
 [ 2.0067    0.6189    1.339228]
 [-2.2755   -0.9234    4.33422 ]
 [ 0.2124   -1.2828    5.853012]
 [ 2.457    -0.9462    4.044788]
 [ 2.1423   -0.453     1.439416]
 [-1.5087   -2.2443    2.459204]
 [ 0.1119   -2.7504    3.264096]
 [ 1.5156   -2.028     2.245276]
 [ 1.4811   -1.0794    0.90508 ]
 [-0.7251   -1.7079    0.771496]
 [-0.0306   -1.9908    1.04302 ]
 [ 0.6558 

208

In [236]:
bx7 = []
by7 = []
bz7 = []
for x in range(len(bx)):
    if x % 16 == 6:
        bx7.append(bx[x])
        by7.append(by[x])
        bz7.append(bz[x])
print(bz7)
print(pos)

[5.486624, 4.044788, 2.801392, 2.028928, 1.6475360000000001, 1.554608, 1.74724, 2.2539879999999997, 3.223924, 4.60768, 5.9749799999999995, 6.395092, 5.496788]
[[ 6.75 -6.75  0.  ]
 [ 2.25 -6.75  0.  ]
 [-2.25 -6.75  0.  ]
 [-6.75 -6.75  0.  ]
 [ 6.75 -2.25  0.  ]
 [ 2.25 -2.25  0.  ]
 [-2.25 -2.25  0.  ]
 [-6.75 -2.25  0.  ]
 [ 6.75  2.25  0.  ]
 [ 2.25  2.25  0.  ]
 [-2.25  2.25  0.  ]
 [-6.75  2.25  0.  ]
 [ 6.75  6.75  0.  ]
 [ 2.25  6.75  0.  ]
 [-2.25  6.75  0.  ]
 [-6.75  6.75  0.  ]]


In [237]:
ts=(np.linspace(0, 360, 13))
print(ts)
position = [(5*np.cos((t-120)*np.pi/180), 5*np.sin((t-120)*np.pi/180), 16.86) for t in ts]
print(position)
cylinder = magpy.magnet.Cylinder(polarization=(0,0,M0), dimension=(shape[0]*2, shape[1]))
cylinder.position = position
cylinder.position
sim = np.array(cylinder.getB([2.25, 2.25, 0]))
print(sim)
sim[:,0]


[  0.  30.  60.  90. 120. 150. 180. 210. 240. 270. 300. 330. 360.]
[(-2.499999999999999, -4.330127018922194, 16.86), (3.061616997868383e-16, -5.0, 16.86), (2.5000000000000004, -4.330127018922193, 16.86), (4.330127018922194, -2.4999999999999996, 16.86), (5.0, 0.0, 16.86), (4.330127018922194, 2.4999999999999996, 16.86), (2.5000000000000004, 4.330127018922193, 16.86), (3.061616997868383e-16, 5.0, 16.86), (-2.499999999999999, 4.330127018922194, 16.86), (-4.330127018922194, 2.4999999999999996, 16.86), (-5.0, 6.123233995736766e-16, 16.86), (-4.330127018922193, -2.5000000000000004, 16.86), (-2.500000000000002, -4.330127018922193, 16.86)]
[[-2.48992923 -3.44927381  5.57135361]
 [-1.24312389 -4.00562142  5.96777659]
 [ 0.1516201  -3.99071818  6.73336129]
 [ 1.40980844 -3.21931788  7.7616444 ]
 [ 2.05841176 -1.68415508  8.80034304]
 [ 1.6510759   0.19843451  9.4726706 ]
 [ 0.19843451  1.6510759   9.4726706 ]
 [-1.68415508  2.05841176  8.80034304]
 [-3.21931788  1.40980844  7.7616444 ]
 [-3.99071

array([-2.48992923, -1.24312389,  0.1516201 ,  1.40980844,  2.05841176,
        1.6510759 ,  0.19843451, -1.68415508, -3.21931788, -3.99071818,
       -4.00562142, -3.44927381, -2.48992923])

In [255]:
bx[0]

-2.4072

In [251]:
index = np.array(range(len(bx7)))
measuredbx = go.Scatter(x=index, y=bx7, mode='markers')
simbx = go.Scatter(x=index, y=sim[:,0], mode='markers')
layoutbx = go.Layout(title='b_x over time')
figbx = go.Figure(data=[measuredbx], layout=layoutbx)
figbx.update_yaxes(scaleanchor='y')
figbx.update_layout(
    xaxis_title="Index",
    yaxis_title="b_x [mT]"
)
figbx.show()

In [252]:
measuredby = go.Scatter(x=index, y=by7, mode='markers')
simby = go.Scatter(x=index, y=sim[:,1], mode='markers')
layoutby = go.Layout(title='b_y over time')
figby = go.Figure(data=[measuredby], layout=layoutby)
figby.update_yaxes(scaleanchor='y')
figby.update_layout(
    xaxis_title="Index",
    yaxis_title="b_y [mT]"
)
figby.show()

In [253]:
measuredbz = go.Scatter(x=index, y=bz7, mode='markers')
simbz = go.Scatter(x=index, y=sim[:,1], mode='markers')
layoutbz = go.Layout(title='b_z over time')
figbz = go.Figure(data=[measuredbz], layout=layoutbz)
figbz.update_yaxes(scaleanchor='y')
figbz.update_layout(
    xaxis_title="Index",
    yaxis_title="b_z [mT]"
)
figbz.show()

In [241]:
size = 2

measured = go.Cone(x=locpred[:,0], y=locpred[:,1], z=locpred[:,2], u=anglepred[:,0], v=anglepred[:,1], w=anglepred[:,2], 
                             sizemode="absolute", sizeref=size, )

#calculated = go.Cone(x=locact[:,0], y=locact[:,1], z=locact[:,2], u=actang[:,0], v=actang[:,1], w=actang[:,2], 
#                             sizemode="absolute", sizeref=size, colorscale=[(0, 'blue'), (1, 'blue')])

layout = go.Layout(title='Location Prediction', scene=dict(aspectmode='data', xaxis_title='x [mm]', yaxis_title='y [mm]', zaxis_title='z [mm]'))

fig = go.Figure(data=[measured], layout=layout)

fig.show()

In [242]:
a = 0
b = 1

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=locpred[:,2], mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Index"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='Rotation over Time', scene=dict(xaxis_title='x [mm]', yaxis_title='y [mm]', zaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [243]:
a = 0
b = 2

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=np.array(range(len(locpred[:,a]))), mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Colorbar"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='x vs z', scene=dict(xaxis_title='x [mm]', yaxis_title='z [mm]', zaxis_title='Index'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [244]:
a = 1
b = 2

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=np.array(range(len(locpred[:,a]))), mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Index"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='y vs z', scene=dict(xaxis_title='y [mm]', yaxis_title='z [mm]', zaxis_title='Index'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [245]:
a = 0
b = 1

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='x vs y (measured)')
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="y [mm]"
)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [246]:
a = 0
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='x vs z (measured)', scene=dict(xaxis_title='x [mm]', yaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="z [mm]"
)
fig2.show()

In [247]:
a = 1
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='y vs z (measured)', scene=dict(xaxis_title='y [mm]', yaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.update_layout(
    xaxis_title="y [mm]",
    yaxis_title="z [mm]"
)
fig2.show()